In [41]:
from pathlib import Path
from IPython import embed

root = Path(r"I:\Cryptography\result")

type_map = {
    'sample_random': "随机",
    "sample_high_density": "高密",
    "sample_low_density": "低密"
}

def extract(lines: list):
    # print(''.join(lines))
    lines = lines[7:]
    lines = map(str.split, map(str.strip, lines))
    lines = [' '.join(l).replace(' *', '*').split() for l in lines]
    checks = []

    for l in lines:
        if len(l) == 0:
            break
        # print(l)
        test_name = l[-1]
        ratio = l[-2]
        pv = l[-3]
        checks += [(test_name, ratio, pv, '*' not in ratio + pv)]
    return checks

result = {}
for d in root.iterdir():
    if d.is_dir():
        for t in d.iterdir():
            if t.is_dir():
                for res in t.iterdir():
                    with open(res, "r") as f:
                        cs = extract(f.readlines())
                    for c in cs:
                        if c[0] not in result:
                            result[c[0]] = [(d.name, t.name, *c[1:])]
                        else:
                            result[c[0]] += [(d.name, t.name, *c[1:])]

def table(name):
    non_pass = False

    for rec in result[name]:
        if not rec[4]:
            if not non_pass:
                print("以下测试点存在异常：\n\n|数据规模|输入类型|通过率|P值|\n|---|---|---|---|")
            print(f"|{rec[0]}|{type_map[rec[1]]}|{rec[2]}|{rec[3]}|")
            non_pass = True
    if non_pass:
        non_pass = False
        print("\n其余测试点全部通过。")
        for rec in result[name]:
            if rec[4]:
                if not non_pass:
                    print("\n\n|数据规模|输入类型|通过率|P值|\n|---|---|---|---|")
                print(f"|{rec[0]}|{type_map[rec[1]]}|{rec[2]}|{rec[3]}|")
                non_pass = True
    else:
        print("有效测试点全部通过。\n\n|数据规模|输入类型|通过率|P值|\n|---|---|---|---|")
        for rec in result[name]:
            print(f"|{rec[0]}|{type_map[rec[1]]}|{rec[2]}|{rec[3]}|")
    print()


def report_gen():
    for k in sorted(result.keys()):
        print("####", k)
        print()
        if k == 'NonOverlappingTemplate':
            print("Non-Overlapping Template Test 有众多匹配模板，对每种 $(数据规模，输入类型)$ 组合，考察测试通过率和P值均一性测试通过率，列举如下。\n\n|数据规模|输入类型|检测通过比例|P值通过比例|\n|---|---|---|---|")
            t0, t1 = '', ''
            pass_c, pv_c, total = 0, 0, 0
            for rec in result[k]:
                if rec[0] != t0 or rec[1] != t1:
                    if len(t0) and len(t1):
                        print(f"|{t0}|{type_map[t1]}|{pass_c}/{total}|{pv_c}/{total}|")
                    t0, t1 = rec[:2]
                    pass_c, pv_c, total = 0, 0, 0
                if '*' not in rec[2]:
                    pass_c += 1
                if '*' not in rec[3]:
                    pv_c += 1
                total += 1
            print()
        else:
            table(k)


In [ ]:
report_gen()